<a href="https://colab.research.google.com/github/philippengani/shoppy/blob/main/shoppy_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pyspark import SparkConf, SparkContext, sql
from pyspark.sql import SparkSession

ModuleNotFoundError: ignored

In [ ]:
# Creating the database connection to the shoppy mysql database

jdbcHostname = "novobyte.org"
jdbcDatabase = "jsvx9284_shpy"
username = 'jsvx9284_ndserge'
password = 'DX%A818lF.+]'
jdbcPort = 3306
jdbcUrl = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}".format(jdbcHostname, jdbcPort, jdbcDatabase, username, password)



In [ ]:
# Importing the shoppy ratings data

shoppy_ratings_df = spark.read.jdbc(url=jdbcUrl, table="shoppy_ratings")
shoppy_ratings_df

Out[12]: DataFrame[rating_id: bigint, rating_value: double, user_id: bigint, reference_id: bigint, type: string]

In [ ]:
display(shoppy_ratings_df)

NameError: ignored